In [46]:
import pandas as pd
import re
from tqdm import tqdm 

In [47]:
from fuzzywuzzy import fuzz
from difflib import SequenceMatcher

# Prize winners in HathiTrust titlemeta.tsv

In [48]:
"""
let's open our data
+
keep only the columns with which we gonna work
"""

titlemeta = pd.read_csv("titlemeta.tsv", sep="\t", low_memory=False)
titlemeta_small = titlemeta[['author', 'title', 'docid']].drop_duplicates(['author', 'title']).reset_index(drop=True)

prizes = pd.read_csv("prizes.tsv", sep="\t", low_memory=False)

In [49]:
"""
keep only the columns with which we gonna work
"""

titlemeta_small.shape

(123371, 3)

In [50]:
"""
clean the titles from everything what follows the mojibake '| $c:' 
+
let's crerate a new column with clean titles
"""

clean_titles = []
for index, row in titlemeta_small.iterrows():
    if re.findall(" \| \$c:", row["title"]):
        slice_index = row["title"].rfind("| $c:")
        clean_title = re.sub("[\/\|\[\]]", '', row["title"][:slice_index])
        clean_titles.append(clean_title)
    else:
        clean_title = re.sub("[\/\|\[\]]", '', row["title"][:slice_index])
        clean_titles.append(clean_title)

        
titlemeta_small["clean_title"] = clean_titles
        

In [51]:
titlemeta_small

,author,title,docid,clean_title
0,"Spencer, Louise Reid",Guerrilla wife | $c: [by] Louise Reid Spencer.,mdp.39015031913893,Guerrilla wife
1,"Baker, Robert H",The suburbs : | a novel / | $c: by Robert H. B...,mdp.39015003936864,The suburbs : a novel
2,"Dickens, Charles",Edwin Drood. | $c: By Charles Dickens. With il...,mdp.39015068342305,Edwin Drood.
3,"Stretton, Hesba","Carola, | $c: by Hesba Stretton.",mdp.39015055066586,"Carola,"
4,"Stretton, Hesba",In prison & out. | $c: By Hesba Stretton.,mdp.39015055066594,In prison & out.
...,...,...,...,...
123366,NaN,Six plays of the yiddish theatre / | $c: [tran...,umn.31951d030369342,Six plays of the yiddish theatre
123367,"Wison, Steven E. (Steven Eugene)",The ghosts of Anatolia : | an epic journey to ...,mdp.39076002906829,The ghosts of Anatolia : an epic journey to f...
123368,"Jeon, Heecheon",Subjectivity of différance : | a poiesis of de...,inu.30000127716409,Subjectivity of différance : a poiesis of dec...
123369,"Wells, H. G. (Herbert George)",Certain personal matters / | $c: by H. G. Wells.,ien.35556042103887,Certain personal matters


In [57]:
"""
let's do the fuzzy matching for the HT dataframe and another df
+
create a new dataframe with the intersections
NB: the code is partly borrowed from Wenyi's notebook
"""

def dataframes_match(titlemeta, df_to_compare, df_author, df_title,
                    n_ratio_author, n_ratio_title):

    titlemeta_ID = []
    titlemeta_author = []
    titlemeta_title = []
    prize_author = []
    prize_title = []
    ratio_author_lst = []
    ratio_title_lst = []

    """
    perhaps, it's better to create another algorythm, loop in loop works very slowly -> maybe lambda can 
    work better
    """

    for idx, row in tqdm(titlemeta.iterrows()):

        author = str(row["author"]).lower()
        title = str(row["clean_title"]).lower()

        if title.startswith('the '):        
            title = title[4:]               


        for idx2, row2 in df_to_compare.iterrows():
            author2 = str(row2["author"]).lower()
            title2 = str(row2["title"]).lower()

            if title2.startswith('the '):
                title2 = title2[4:]

            ratio_author = fuzz.ratio(author2, author)
            ratio_title = fuzz.ratio(title2, title)

            if ratio_author > n_ratio_author and ratio_title > n_ratio_title:


                titlemeta_ID.append(row["docid"])
                titlemeta_author.append(row["author"])
                titlemeta_title.append(row["clean_title"])
                prize_author.append(row2["author"])
                prize_title.append(row2["title"])
                ratio_author_lst.append(ratio_author)
                ratio_title_lst.append(ratio_title)

            else:
                continue

    final_df = pd.DataFrame(list(zip(titlemeta_ID, titlemeta_author, titlemeta_title, prize_author,
                                      prize_title, ratio_author_lst, ratio_title_lst)), columns=["titlemeta_ID", "titlemeta_author", "titlemeta_title", 
                                                                                         df_author, df_title, "ratio_author", "ratio_title"])
    return final_df

In [63]:
hathi_prize = dataframes_match(titlemeta_small, prizes, "prize_author", "prize_title",
                               n_ratio_author = 90, n_ratio_title = 0)

123371it [13:49, 148.68it/s]


In [76]:
"""
let's save our data to csv file
"""

hathi_prize.to_csv('list_of_matches_HT_prize.csv', sep = '\t', index = False)

In [65]:
"""
let's filter two types of authors: those, who got the Prize for the whole work and those, who got the
Prize for a particular work. The ratio of the title is important for us. 
Perhaphs, the ratio_title should be lower than 50, I set it just according to my very subjective opinion
"""
any_book = hathi_prize[hathi_prize['prize_title'] == '#matchall'].reset_index(drop=True)
specific_books = hathi_prize[hathi_prize['prize_title'] != '#matchall'].reset_index(drop=True)
specific_books = specific_books[specific_books['ratio_title'] > 50].reset_index(drop=True)

In [66]:
any_book

,titlemeta_ID,titlemeta_author,titlemeta_title,prize_author,prize_title,ratio_author,ratio_title
0,wu.89099437782,"France, Anatole",The authorized English translations of the nov...,"France, Anatole",#matchall,100,13
1,nyp.33433074788724,"Man, Thomas","A picture of Woonsocket; or, The truth in its...","Mann, Thomas",#matchall,96,12
2,uc2.ark+=13960=t12n5280j,"France, Anatole",The crime of Sylvestre,"France, Anatole",#matchall,100,14
3,mdp.39015030370020,"France, Anatole","Tales from a mother-of-pearl casket,","France, Anatole",#matchall,100,26
4,uc2.ark+=13960=t88g8jv5k,"Pontoppidan, Henrik",Emanuel,"Pontoppidan, Henrik",#matchall,100,38
...,...,...,...,...,...,...,...
326,uc1.32106018935491,"Faulkner, William","Novels, 1926-1929","Faulkner, William",#matchall,100,7
327,mdp.39015063352317,"Deledda, Grazia",Marianna Sirca,"Deledda, Grazia",#matchall,100,32
328,mdp.39015064760872,"Hesse, Hermann",Siddhartha : an Indian poem,"Hesse, Hermann",#matchall,100,21
329,uc1.32106018613668,"Bunin, Ivan Alekseevich",Night of denial : stories and novellas,"Bunin, Ivan Alekseyevich",#matchall,98,20


In [67]:
"""
let's recheck the intersections between titles and get rid off books which are not in the prize_title column
"""

for index, row in specific_books.iterrows():
    if str(row["prize_title"]).lower() not in str(row["titlemeta_title"]).lower():
        specific_books.drop(index, inplace=True)

In [68]:
"""
let's concat two our dataframes and save them as an independenct .csv file
"""
prize_winners = pd.concat([any_book, specific_books]).reset_index(drop=True)
prize_winners.to_csv('match_prize_winners.csv', index = False)

# Bestsellers in HathiTrust titlemeta.tsv

In [69]:
bestsellers = pd.read_csv("new_unsworth_bestsellers.csv", low_memory=False)

In [70]:
bestsellers

,year,author,title
0,1900,"Johnston, Mary",To Have and To Hold
1,1900,"Cholmondeley, Mary",Red Pottage
2,1900,"Grant, Robert",Unleavened Bread
3,1900,"Allen, James Lane",The Reign of Law
4,1900,"Bacheller, Irving",Eben Holden
...,...,...,...
1116,1999,"Fitch, Janet",White Oleander
1117,1999,"Sparks, Nicholas",A Walk to Remember
1118,1999,"Patterson, James",Pop Goes the Weasel
1119,1999,"Cornwell, Patricia",Black Notice


In [71]:
hathi_bestsellers = dataframes_match(titlemeta_small, bestsellers, "bestseller_author", "bestseller_title",
                               n_ratio_author = 90, n_ratio_title = 50)

123371it [5:11:16,  6.61it/s]


In [72]:

hathi_bestsellers.to_csv('list_of_matches_HT_bestsellers.csv', index = False)

In [75]:
hathi_bestsellers[hathi_bestsellers['ratio_title'] > 55].reset_index(drop=True)
# specific_books[specific_books['ratio_title'] > 50].reset_index(drop=True)

,titlemeta_ID,titlemeta_author,titlemeta_title,bestseller_author,bestseller_title,ratio_author,ratio_title
0,mdp.39015063548997,"Glyn, Elinor",Visits of Elizabeth,"Glyn, Elinor",The Visits of Elizabeth,100,97
1,nyp.33433082272703,"Cholmondeley, Mary","Red pottage,","Cholmondeley, Mary",Red Pottage,100,92
2,uc2.ark+=13960=t7gq6sf0x,"Allen, James Lane",The reign of law,"Allen, James Lane",The Reign of Law,100,100
3,mdp.39015005148344,"Grant, Robert",Unleavened bread : a novel,"Grant, Robert",Unleavened Bread,100,71
4,nyp.33433076089436,"Goss, Charles Frederic",The redemption of David Corson,"Goss, Charles Frederic",The Redemption of David Corson,100,96
...,...,...,...,...,...,...,...
762,mdp.39015058795934,"Grafton, Sue",R is for ricochet,"Grafton, Sue",K is for Killer,100,59
763,inu.30000094835844,"Asimov, Isaac",Robot dreams,"Asimov, Isaac",The Robots of Dawn,100,57
764,mdp.39015063688058,"Grafton, Sue",S is for silence,"Grafton, Sue",K is for Killer,100,67
765,pst.000065617298,"Grey, Zane",To the last man,"Grey, Zane",To the Last Man,100,94
